<span>
<b>Author:</b> <a href="http://pages.di.unipi.it/ruggieri/">Salvatore Ruggieri</a><br/>
<b>Python version:</b>  3.x<br/>
</span>

In [1]:
# if using Colab
try:
    import google.colab
    is_colab = True
    wdir = 'https://raw.githubusercontent.com/ruggieris/DD/main/'
    # required modules
    !pip install lime
    !pip install dalex
    !pip install shap
    !pip install pydotplus
except:
    is_colab = False
    wdir = '../' # local files
print('Working dir: ', wdir)

Working dir:  ../


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import pydotplus
#import graphviz
import os

# add if Graphviz is not already in the path
if True:
    os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'

ModuleNotFoundError: No module named 'pydotplus'

In [ ]:
# Adult dataset
adult = pd.read_csv(wdir+"data/adult_continuous.csv", sep=',', na_values='?')
# remove columns
del adult['fnlwgt'] 
del adult['native-country'] 
# impute missing/outlier values
adult['workclass'] = adult['workclass'].fillna(adult['workclass'].mode()[0])
adult['occupation'] = adult['occupation'].fillna(adult['occupation'].mode()[0])
adult.loc[ adult['capital-gain']==99999, 'capital-gain']= int(adult['capital-gain'].mean())
# target class
target = 'class'
adult.head()

In [ ]:
# Encode categorical values into numbers
from sklearn.preprocessing import LabelEncoder

cat = adult.select_dtypes('object').columns
df = pd.DataFrame()
encoders = dict()
for col in adult.columns:
    if col in cat:
        col_encoder = LabelEncoder()
        df[col] = col_encoder.fit_transform(adult[col])
        df[col] = df[col].astype('category')
        encoders[col] = col_encoder
    else:
        df[col] = adult[col]
print(encoders['class'].classes_)
# categorical and numerical and predictive atts 
categorical = [c for c in cat if c!=target]
cat2pos = {f:i for i, f in enumerate(df.columns) if f in categorical} 
categorical_pos = list(cat2pos.values()) # categorical variable positions
numerical = [c for c in df.columns if c!=target and c not in set(categorical)]
atts = [c for c in df.columns if c!=target]
df.head()

In [ ]:
# one hot encoding
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

preprocess = ColumnTransformer([("enc", OneHotEncoder(), categorical_pos)], remainder = 'passthrough')

preprocess.fit(df[atts])
# decode back column names
def mapf(f):
    if f[:3]=='rem':
        return f[11:]
    f = f[5:]
    pos = f.find('_')
    return f[:pos]+'='+encoders[f[:pos]].classes_[int(f[pos+1:])]


print(preprocess.get_feature_names_out())
fnames = [mapf(f) for f in preprocess.get_feature_names_out() if f!=target]
print(fnames)

In [ ]:
# black box model
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import make_pipeline

bb = make_pipeline(preprocess, GradientBoostingClassifier(n_estimators=100))

# Part I: Global model explanations

In [ ]:
# training-test split 60%-40%
from sklearn.model_selection import train_test_split

X = df.drop([target], axis=1)
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

In [ ]:
# black box training
from sklearn.metrics import accuracy_score, f1_score

bb.fit(X_train.values, y_train) # .values for numpy as to prevent warning later on
y_pred = bb.predict(X_test.values)
print('Accuracy: {:.4f}'.format(accuracy_score(y_test, y_pred)))
print('F1-score: {:.4f}'.format(f1_score(y_test, y_pred)))

In [ ]:
# surrogate model assuming to know the training data
from sklearn.tree import DecisionTreeClassifier

sm = make_pipeline(preprocess, DecisionTreeClassifier(max_depth=3))
sm.fit(X_train, bb.predict(X_train))
sm_pred = sm.predict(X_test)
print('Accuracy: {:.4f}'.format(accuracy_score(y_test, sm_pred)))
print('F1-score: {:.4f}'.format(f1_score(y_test, sm_pred)))
print('Fidelity: {:.4f}'.format(accuracy_score(y_pred, sm_pred)))

In [ ]:
# visualize surrogate model
from sklearn import tree
from IPython.display import Image

dot_data = tree.export_graphviz(sm[1], out_file=None, feature_names=fnames, 
             class_names=encoders['class'].classes_, filled=True, rounded=True)  
graph = pydotplus.graph_from_dot_data(dot_data)  
Image(graph.create_png())

In [ ]:
# surrogate model assuming NOT to know the training data
# use half of the test data for training surrogate model
half = int(len(X_test)/2) 
X_surr_train = X_test[:half]
X_surr_test = X_test[half:]
sm.fit(X_surr_train, bb.predict(X_surr_train))

sm_pred = sm.predict(X_surr_test)
print('Accuracy: {:.4f}'.format(accuracy_score(y_test[half:], sm_pred)))
print('F1-score: {:.4f}'.format(f1_score(y_test[half:], sm_pred)))
print('Fidelity: {:.4f}'.format(accuracy_score(y_pred[half:], sm_pred)))

In [ ]:
# visualize surrogate model
dot_data = tree.export_graphviz(sm[1], out_file=None, feature_names=fnames,  
             class_names=encoders['class'].classes_, filled=True, rounded=True)  
graph = pydotplus.graph_from_dot_data(dot_data)  
Image(graph.create_png())

# Part II: Outcome explanations

In [ ]:
# LIME
from lime.lime_tabular import LimeTabularExplainer

# LimeTabularExplainer 
lime_explainer = LimeTabularExplainer(X_test.to_numpy(), # numpy dataset
                    feature_names=X_test.columns, # column names
                    class_names=encoders['class'].classes_,  # class names
                    categorical_features=categorical_pos,
                    categorical_names={cat2pos[f]:encoders[f].classes_ for f in categorical}, # categorical variable value names
                    discretize_continuous=False)

In [ ]:
# explain instance
x = X_test.iloc[3]
print(x)
predict_fn = lambda x: bb.predict_proba(x).astype(float)
exp = lime_explainer.explain_instance(x, predict_fn)
# as attribute, weight
exp.local_exp

In [ ]:
exp.show_in_notebook()

In [ ]:
fig = exp.as_pyplot_figure()

In [ ]:
# Break-down plots
import dalex as dx

exp = dx.Explainer(bb, X_train, y_train)

In [ ]:
# explaning an instance
x = X_test.iloc[3].values
bd_inst = exp.predict_parts(x, type='break_down')
# plotted as attribute=<int>, to get attribute=<value> need to re-train on adult instead of df
bd_inst.plot()

In [ ]:
# SHAP using Dalex
shap = exp.predict_parts(x, type = 'shap', B=5)
shap.plot()

In [ ]:
# SHAP package
import shap

shap.initjs()
predict_fn = lambda x: bb.predict_proba(x)[:, 1]
med = np.median(X_test, axis=0).reshape((1, X_test.shape[1]))
shap_explainer = shap.KernelExplainer(predict_fn, med)

In [ ]:
x = X_test.iloc[3].values
shap_values_single = shap_explainer.shap_values(x)
shap.force_plot(shap_explainer.expected_value, shap_values_single, features=x, feature_names=X_test.columns)

In [ ]:
shap_values = shap_explainer(X_test.iloc[3:4])
shap.plots.waterfall(shap_values[0])

In [ ]:
shap.decision_plot(shap_explainer.expected_value, shap_explainer.shap_values(X_test[:1000]), X_test.columns, ignore_warnings=True)

In [ ]:
# Ceteris Paribus plots using Dalex
cp = exp.predict_profile(x)
cp.plot(variables=numerical, variable_type = "numerical")
cp.plot(variables=categorical, variable_type = "categorical")

# Part III: Model inspection

In [ ]:
# model inspection: PDP
from sklearn.inspection import PartialDependenceDisplay
# age
PartialDependenceDisplay.from_estimator(bb, X_test, ['age'])

In [ ]:
# ICE
PartialDependenceDisplay.from_estimator(bb, X_test, ['age'], kind='both')

In [ ]:
# PDP plot using DALEX
pdp = exp.model_profile(type='partial', variables=['age'])
pdp.plot()